# 第 8 章：卷积神经网络 — TensorFlow

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import matplotlib.pyplot as plt

## 载入数据集

In [2]:
mnist = input_data.read_data_sets('./datasets/ch08/tensorflow/MNIST',one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./datasets/ch08/tensorflow/MNIST\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./datasets/ch08/tensorflow/MNIST\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./datasets/ch08/tensorflow/MNIST\t10k-images-idx3-ubyte.gz
Extracting ./datasets/ch08/tensorflow/MNIST\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
print(mnist.train.images.shape)
print(mnist.train.labels.shape)

(55000, 784)
(55000, 10)


In [4]:
print(mnist.validation.images.shape)
print(mnist.validation.labels.shape)

(5000, 784)
(5000, 10)


In [5]:
print(mnist.test.images.shape)
print(mnist.test.labels.shape)

(10000, 784)
(10000, 10)


## 定义创建模型需要的函数

In [6]:
# input 代表输入，filter 代表卷积核
# 卷积层
def conv2d(x, filter):
    return tf.nn.conv2d(x, 
                        filter, 
                        strides=[1,1,1,1], 
                        padding='SAME')

# 池化层
def max_pool(x):
    return tf.nn.max_pool(x, 
                          ksize=[1,2,2,1], 
                          strides=[1,2,2,1], 
                          padding='SAME')

# 初始化卷积核或者是权重数组的值
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

# 初始化bias的值
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

## 定义占位符 placeholder

In [7]:
# None　代表图片数量未知
x = tf.placeholder(tf.float32, [None,784])/255
# 将input 重新调整结构，适用于CNN的特征提取
x_image = tf.reshape(x, [-1,28,28,1])

# y是最终预测的结果
y = tf.placeholder(tf.float32, [None,10])

## 定义 CNN 模型

### CONV1

In [8]:
W_conv1 = weight_variable([5, 5, 1, 32])  # 卷积核尺寸：5x5, 输入通道：1, 输出通道：32
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)  # 输出尺寸：28x28x32
h_pool1 = max_pool(h_conv1)     # 输出尺寸：14x14x32

### CONV2

In [9]:
W_conv2 = weight_variable([5, 5, 32, 64])  # 卷积核尺寸：5x5, 输入通道：32, 输出通道：64
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)  # 输出尺寸：14x14x64
h_pool2 = max_pool(h_conv2)     # 输出尺寸：7x7x64

### FC1

In [10]:
W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])  # 展开为一维向量
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

### FC2

In [11]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
prediction = tf.nn.softmax(tf.matmul(h_fc1, W_fc2) + b_fc2)

## 定义损失函数与优化算法

In [12]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y * tf.log(prediction),reduction_indices=[1]))   
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [13]:
#判断预测标签和实际标签是否匹配
correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float"))

## 训练并验证准确率

In [14]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

cost = []

for i in range(1000):
    batch_x, batch_y = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_x, y: batch_y})
    if (i+1) % 50 == 0:
        print("train accuracy %.3f" % accuracy.eval(session = sess,
                feed_dict = {x:batch_x, y:batch_y}))
print("test accuracy %.3f" % accuracy.eval(session = sess,
    feed_dict = {x:mnist.test.images, y:mnist.test.labels}))

train accuracy 0.810
train accuracy 0.890
train accuracy 0.880
train accuracy 0.960
train accuracy 0.950
train accuracy 0.940
train accuracy 0.950
train accuracy 0.940
train accuracy 0.940
train accuracy 0.960
train accuracy 0.980
train accuracy 0.950
train accuracy 0.940
train accuracy 0.980
train accuracy 0.980
train accuracy 0.980
train accuracy 1.000
train accuracy 0.990
train accuracy 0.990
train accuracy 0.990
test accuracy 0.977
